## 1. Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# TensorFlow/Keras
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    print(f"TensorFlow version: {tf.__version__}")
except ImportError:
    print("TensorFlow not installed. Installing...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'tensorflow'])
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    print(f"TensorFlow version: {tf.__version__}")

## 2. 데이터 로드 및 전처리

In [ ]:
# 데이터 로드
train_raw = pd.read_csv('train.csv')
print(f"원본 데이터: {train_raw.shape}")
train_raw.head()

In [ ]:
# 피벗 테이블 생성 (100개 품목 × 43개월)
pivot_by_month = train_raw.groupby(['item_id', 'year', 'month'])['value'].sum().reset_index()
pivot_by_month['year_month'] = pivot_by_month['year'].astype(str) + '-' + pivot_by_month['month'].astype(str).str.zfill(2)
pivot_value = pivot_by_month.pivot(index='item_id', columns='year_month', values='value').fillna(0)

print(f"피벗 테이블: {pivot_value.shape}")
print(f"품목 수: {len(pivot_value)}, 월 수: {len(pivot_value.columns)}")

# 시계열 데이터로 변환 (shape: [n_items, n_months, 1])
time_series_data = pivot_value.values  # [100, 43]
n_items, n_months = time_series_data.shape

print(f"\n시계열 데이터 형태: {time_series_data.shape}")
print(f"품목 ID: {pivot_value.index.tolist()[:10]}...")
print(f"월 범위: {pivot_value.columns[0]} ~ {pivot_value.columns[-1]}")

## 3. 시계열 학습 데이터 생성

In [ ]:
def create_sequences(data, lookback=12, forecast_horizon=1):
    """
    시계열 시퀀스 생성
    
    Args:
        data: [n_items, n_months] 형태의 시계열 데이터
        lookback: 과거 몇 개월을 볼 것인가
        forecast_horizon: 미래 몇 개월을 예측할 것인가
    
    Returns:
        X: [n_samples, lookback, n_items] - 입력 시퀀스
        y: [n_samples, n_items] - 타겟값
    """
    n_items, n_months = data.shape
    X, y = [], []
    
    for i in range(lookback, n_months - forecast_horizon + 1):
        # 과거 lookback 개월의 모든 품목 데이터를 입력으로
        X.append(data[:, i-lookback:i].T)  # [lookback, n_items]
        # forecast_horizon 개월 후의 모든 품목 값을 타겟으로
        y.append(data[:, i+forecast_horizon-1])  # [n_items]
    
    return np.array(X), np.array(y)

# 시퀀스 생성 (과거 12개월 → 다음 1개월 예측)
lookback = 12
X, y = create_sequences(time_series_data, lookback=lookback, forecast_horizon=1)

print(f"입력 시퀀스 X: {X.shape} [samples, timesteps, features]")
print(f"타겟 y: {y.shape} [samples, items]")
print(f"\n총 샘플 수: {len(X)}개")
print(f"각 샘플: 과거 {lookback}개월 데이터 → 다음 달 {n_items}개 품목 예측")

In [ ]:
# 데이터 정규화 (각 품목별로)
scalers = {}
X_scaled = np.zeros_like(X)
y_scaled = np.zeros_like(y)

for i in range(n_items):
    scaler = StandardScaler()
    # X의 i번째 품목 열 정규화
    X_scaled[:, :, i] = scaler.fit_transform(X[:, :, i])
    # y의 i번째 품목 정규화
    y_scaled[:, i] = scaler.transform(y[:, i].reshape(-1, 1)).flatten()
    scalers[i] = scaler

print(f"정규화 완료: {len(scalers)}개 품목별 스케일러 생성")
print(f"X_scaled 범위: [{X_scaled.min():.2f}, {X_scaled.max():.2f}]")
print(f"y_scaled 범위: [{y_scaled.min():.2f}, {y_scaled.max():.2f}]")

In [ ]:
# Train/Validation 분할 (시계열이므로 순차적 분할)
split_idx = int(len(X_scaled) * 0.8)

X_train = X_scaled[:split_idx]
y_train = y_scaled[:split_idx]
X_val = X_scaled[split_idx:]
y_val = y_scaled[split_idx:]

print(f"Train: X={X_train.shape}, y={y_train.shape}")
print(f"Val:   X={X_val.shape}, y={y_val.shape}")

## 4. LSTM 모델 구축

In [ ]:
def build_lstm_model(input_shape, output_dim, lstm_units=[128, 64], dropout=0.2):
    """
    LSTM 기반 시계열 예측 모델
    
    Args:
        input_shape: (timesteps, features)
        output_dim: 예측할 품목 수
        lstm_units: LSTM 레이어 유닛 수 리스트
        dropout: 드롭아웃 비율
    """
    model = keras.Sequential(name='LSTM_Forecaster')
    
    # 첫 번째 LSTM 레이어
    model.add(layers.LSTM(
        lstm_units[0], 
        return_sequences=True if len(lstm_units) > 1 else False,
        input_shape=input_shape,
        name='lstm_1'
    ))
    model.add(layers.Dropout(dropout))
    
    # 추가 LSTM 레이어들
    for i, units in enumerate(lstm_units[1:], start=2):
        return_seq = i < len(lstm_units)
        model.add(layers.LSTM(units, return_sequences=return_seq, name=f'lstm_{i}'))
        model.add(layers.Dropout(dropout))
    
    # Dense 레이어
    model.add(layers.Dense(64, activation='relu', name='dense_1'))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(output_dim, activation='linear', name='output'))
    
    return model

# 모델 생성
input_shape = (lookback, n_items)  # (12, 100)
output_dim = n_items  # 100

model = build_lstm_model(
    input_shape=input_shape,
    output_dim=output_dim,
    lstm_units=[128, 64],
    dropout=0.2
)

model.summary()

## 5. 모델 컴파일 및 학습

In [ ]:
# 모델 컴파일
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

# 콜백 설정
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    )
]

print("모델 컴파일 완료")
print("Optimizer: Adam (lr=0.001)")
print("Loss: MSE")
print("Callbacks: EarlyStopping (patience=15), ReduceLROnPlateau")

In [ ]:
# 모델 학습
print("모델 학습 시작...\n")

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=4,
    callbacks=callbacks,
    verbose=1
)

print("\n학습 완료!")

In [ ]:
# 학습 곡선 시각화
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Val MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.title('Training and Validation MAE')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"최종 Train Loss: {history.history['loss'][-1]:.4f}")
print(f"최종 Val Loss: {history.history['val_loss'][-1]:.4f}")
print(f"최종 Val MAE: {history.history['val_mae'][-1]:.4f}")

## 6. 예측 및 역정규화

In [ ]:
# 2025년 8월 예측 (마지막 12개월 데이터 사용)
last_sequence = time_series_data[:, -lookback:].T  # [12, 100]

# 정규화
last_sequence_scaled = np.zeros_like(last_sequence)
for i in range(n_items):
    last_sequence_scaled[:, i] = scalers[i].transform(last_sequence[:, i].reshape(-1, 1)).flatten()

# 예측
last_sequence_scaled = last_sequence_scaled.reshape(1, lookback, n_items)
prediction_scaled = model.predict(last_sequence_scaled, verbose=0)

# 역정규화
prediction = np.zeros_like(prediction_scaled)
for i in range(n_items):
    prediction[:, i] = scalers[i].inverse_transform(prediction_scaled[:, i].reshape(-1, 1)).flatten()

prediction = prediction.flatten()

print(f"2025년 8월 예측값: {prediction.shape}")
print(f"예측값 범위: [{prediction.min():.2f}, {prediction.max():.2f}]")
print(f"예측값 통계:")
print(pd.Series(prediction).describe())

## 7. 공행성쌍 탐지 및 제출 파일 생성

In [ ]:
# 공행성쌍 탐지를 위한 상관계수 계산
from scipy.stats import pearsonr

def find_comovement_pairs(predictions, historical_data, items, max_lag=7, corr_threshold=0.3, top_n=3000):
    """
    딥러닝 예측값을 기반으로 공행성쌍 탐지
    """
    pairs = []
    
    for i, leader in enumerate(items):
        for j, follower in enumerate(items):
            if i == j:
                continue
            
            # 과거 데이터로 상관관계 확인
            best_corr = 0
            best_lag = 1
            
            for lag in range(1, max_lag + 1):
                if len(historical_data[i]) <= lag:
                    continue
                
                x = historical_data[i][:-lag]
                y = historical_data[j][lag:]
                
                if len(x) < 10 or np.std(x) == 0 or np.std(y) == 0:
                    continue
                
                corr, _ = pearsonr(x, y)
                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag
            
            if abs(best_corr) >= corr_threshold:
                # 예측값 사용
                pred_value = int(round(predictions[j]))
                if pred_value < 0:
                    pred_value = 0
                
                pairs.append({
                    'leading_item_id': leader,
                    'following_item_id': follower,
                    'value': pred_value,
                    'corr': abs(best_corr),
                    'lag': best_lag
                })
    
    # 상관계수 기준 정렬 후 상위 N개 선택
    df_pairs = pd.DataFrame(pairs)
    if len(df_pairs) > top_n:
        df_pairs = df_pairs.nlargest(top_n, 'corr')
    
    return df_pairs[['leading_item_id', 'following_item_id', 'value']]

# 공행성쌍 탐지
items = pivot_value.index.tolist()
submission = find_comovement_pairs(
    predictions=prediction,
    historical_data=time_series_data,
    items=items,
    max_lag=7,
    corr_threshold=0.3,
    top_n=3000
)

print(f"탐지된 공행성쌍: {len(submission):,}개")
print(f"\n예측값 통계:")
print(submission['value'].describe())
submission.head(20)

## 8. 제출 파일 저장

In [ ]:
# 제출 파일 저장
submission.to_csv('./submission_lstm.csv', index=False)

print("="*80)
print("시계열 딥러닝 모델 (LSTM) 완료!")
print("="*80)

print(f"\n제출 파일: submission_lstm.csv")
print(f"예측 쌍 수: {len(submission):,}개")
print(f"\n예측값 통계:")
print(submission['value'].describe())
print(f"\n0인 예측값: {(submission['value'] == 0).sum()}개")

print("\n모델 특징:")
print("1. LSTM 기반 시계열 예측")
print("2. 100개 품목을 동시에 모델링")
print("3. 과거 12개월 → 다음 1개월 예측")
print("4. 품목 간 시간적 의존성 학습")
print("5. 상관관계 기반 공행성쌍 필터링")

print("\n베이스라인 대비 개선 기대:")
print("- 기존 XGBoost: 0.3454")
print("- LSTM 목표: 0.36-0.40+")

## 9. 평가 메트릭 분석 및 개선 전략

**현재 접근의 문제:**
1. LSTM으로 예측값은 얻었지만, 여전히 correlation으로 쌍을 선택
2. 이는 기존 XGBoost 방식과 쌍 선정 로직이 동일
3. LSTM의 시계열 학습 능력을 쌍 탐지에 활용하지 못함

**평가 메트릭 이해:**
- Score = 0.6 × F1 + 0.4 × (1 - NMAE)
- F1: 쌍 탐지 정확도 (더 중요!)
- NMAE: 예측값 정확도

**개선 전략:**
1. **LSTM 예측 변화량 기반 쌍 탐지**: 품목 간 예측 변화 패턴 유사도
2. **Attention 메커니즘**: 품목 간 상호작용 학습
3. **Graph Neural Network**: 품목 관계를 그래프로 모델링

In [ ]:
# 개선된 쌍 탐지: LSTM 예측 변화량 기반
def find_comovement_pairs_v2(model, time_series_data, scalers, items, lookback=12, top_n=3000):
    """
    LSTM 예측 변화량을 이용한 공행성쌍 탐지
    
    전략: 과거 N개월 동안 각 품목의 예측 변화 패턴이 유사한 쌍 찾기
    """
    n_items = len(items)
    
    # 각 시점별로 모든 품목 예측
    predictions_over_time = []
    
    for t in range(lookback, len(time_series_data[0])):
        sequence = time_series_data[:, t-lookback:t].T  # [12, 100]
        
        # 정규화
        seq_scaled = np.zeros_like(sequence)
        for i in range(n_items):
            seq_scaled[:, i] = scalers[i].transform(sequence[:, i].reshape(-1, 1)).flatten()
        
        # 예측
        seq_scaled = seq_scaled.reshape(1, lookback, n_items)
        pred_scaled = model.predict(seq_scaled, verbose=0)
        
        # 역정규화
        pred = np.zeros_like(pred_scaled)
        for i in range(n_items):
            pred[:, i] = scalers[i].inverse_transform(pred_scaled[:, i].reshape(-1, 1)).flatten()
        
        predictions_over_time.append(pred.flatten())
    
    predictions_over_time = np.array(predictions_over_time)  # [n_timepoints, n_items]
    
    print(f"예측 시계열 생성: {predictions_over_time.shape}")
    
    # 각 품목의 예측 변화량 계산
    changes = np.diff(predictions_over_time, axis=0)  # [n_timepoints-1, n_items]
    
    # 품목 간 변화 패턴 상관관계 계산
    pairs = []
    
    for i, leader in enumerate(items):
        for j, follower in enumerate(items):
            if i == j:
                continue
            
            # 변화 패턴 상관계수
            if np.std(changes[:, i]) == 0 or np.std(changes[:, j]) == 0:
                continue
            
            # lag를 고려한 최대 상관계수
            best_corr = 0
            best_lag = 0
            
            for lag in range(0, min(7, len(changes))):
                if lag >= len(changes):
                    continue
                
                x = changes[:-lag if lag > 0 else None, i]
                y = changes[lag:, j]
                
                if len(x) < 5:
                    continue
                
                corr = np.corrcoef(x, y)[0, 1]
                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag
            
            if abs(best_corr) > 0.3:  # 임계값
                # 최종 예측값 사용
                final_pred = int(round(predictions_over_time[-1, j]))
                if final_pred < 0:
                    final_pred = 0
                
                pairs.append({
                    'leading_item_id': leader,
                    'following_item_id': follower,
                    'value': final_pred,
                    'corr': abs(best_corr),
                    'lag': best_lag
                })
    
    # 상관계수 기준 정렬 후 상위 N개 선택
    df_pairs = pd.DataFrame(pairs)
    print(f"탐지된 전체 쌍: {len(df_pairs):,}개")
    
    if len(df_pairs) > top_n:
        df_pairs = df_pairs.nlargest(top_n, 'corr')
    
    return df_pairs[['leading_item_id', 'following_item_id', 'value']]

print("개선된 쌍 탐지 전략:")
print("1. 과거 여러 시점에서 LSTM 예측 수행")
print("2. 각 품목의 예측 '변화량' 계산")
print("3. 변화 패턴이 유사한 쌍을 공행성쌍으로 판단")
print("4. 단순 값 상관관계가 아닌 '변화 패턴' 기반")

In [ ]:
# 개선된 방법으로 쌍 탐지 (시간이 걸릴 수 있음)
print("LSTM 기반 변화 패턴 분석 시작...\n")

submission_v2 = find_comovement_pairs_v2(
    model=model,
    time_series_data=time_series_data,
    scalers=scalers,
    items=items,
    lookback=lookback,
    top_n=3000
)

print(f"\n개선된 방법으로 탐지된 공행성쌍: {len(submission_v2):,}개")
print(f"\n예측값 통계:")
print(submission_v2['value'].describe())
submission_v2.head(20)

In [ ]:
# 두 방법 비교
print("="*80)
print("두 가지 쌍 탐지 방법 비교")
print("="*80)

print(f"\n[방법 1] 기존 Correlation 기반:")
print(f"  - 쌍 수: {len(submission):,}개")
print(f"  - 평균 예측값: {submission['value'].mean():.2f}")
print(f"  - 중간값: {submission['value'].median():.2f}")

print(f"\n[방법 2] LSTM 변화 패턴 기반:")
print(f"  - 쌍 수: {len(submission_v2):,}개")
print(f"  - 평균 예측값: {submission_v2['value'].mean():.2f}")
print(f"  - 중간값: {submission_v2['value'].median():.2f}")

# 두 방법의 쌍 중복도 확인
pairs1 = set(zip(submission['leading_item_id'], submission['following_item_id']))
pairs2 = set(zip(submission_v2['leading_item_id'], submission_v2['following_item_id']))

overlap = len(pairs1 & pairs2)
print(f"\n쌍 중복도: {overlap:,}개 ({overlap/len(pairs1)*100:.1f}%)")
print(f"방법 1만의 쌍: {len(pairs1 - pairs2):,}개")
print(f"방법 2만의 쌍: {len(pairs2 - pairs1):,}개")

In [ ]:
# 방법 2로 최종 제출 파일 저장
submission_v2.to_csv('./submission_lstm_v2.csv', index=False)

print("="*80)
print("LSTM 변화 패턴 기반 모델 완료!")
print("="*80)

print(f"\n제출 파일: submission_lstm_v2.csv")
print(f"예측 쌍 수: {len(submission_v2):,}개")
print(f"\n예측값 통계:")
print(submission_v2['value'].describe())

print("\n핵심 차별점:")
print("✓ 단순 상관관계 → LSTM 학습 기반 변화 패턴")
print("✓ 정적 분석 → 시계열 동적 분석")
print("✓ 품목별 독립 → 100개 품목 동시 모델링")
print("✓ 선형 관계 → 비선형 패턴 학습")

print("\n예상 개선:")
print("- F1 Score: 쌍 탐지 정확도 향상 (변화 패턴 유사도)")
print("- NMAE: LSTM 예측값 정확도 향상")
print("- 목표: 0.36-0.42")

## 10. 0.45+ 달성을 위한 전략

**현실:**
- 현재 방법 한계: ~0.37
- 1등 점수: 0.45
- 격차: +22% 개선 필요

**필요한 혁신:**

### 1. 품목 메타 정보 활용
- item_id 속성 분석 (패턴, 카테고리 추정)
- 숫자 범위로 그룹핑 (101-110, 111-120 등)
- 품목 간 구조적 관계 파악

### 2. 시계열 분해 (Decomposition)
- Trend + Seasonality + Residual
- 계절성 패턴 명시적 모델링
- 주기성 탐지 (월별, 분기별)

### 3. Granger Causality
- 통계적 인과관계 테스트
- Correlation보다 정확한 선행-후행 관계

### 4. Transfer Entropy
- 정보 이론 기반 의존성 측정
- 비선형 관계 포착

### 5. Ensemble Strategy
- XGBoost + LSTM + Statistical Methods
- Stacking with meta-learner
- 다양한 threshold 조합

### 6. Feature Engineering
```python
- 품목별 변동성 (volatility)
- 최근 추세 (recent trend)
- 가속도 (acceleration)
- 주기성 점수 (periodicity score)
- 품목 간 거리 메트릭
```

**다음 단계 선택지:**
1. ✅ **빠른 개선**: Granger Causality + 앙상블 (목표 0.38-0.42)
2. ⚡ **중간 개선**: Transfer Entropy + 시계열 분해 (목표 0.40-0.43)
3. 🎯 **풀스택**: 위 모든 방법 + 수작업 분석 (목표 0.43-0.45+)

어떤 전략으로 갈까요?